# Generate some data

In [1]:
from bayes_window.fake_spikes import generate_fake_spikes
from bayes_window.visualization import fake_spikes_explore, plot_data_and_posterior
from bayes_window import models
from bayes_window.fitting import fit_numpyro
from bayes_window.utils import add_data_to_posterior

/home/m/anaconda3/envs/tf/lib/python3.6/site-packages/elephant/pandas_bridge.py:22: DeprecationWarning: pandas_bridge module will be removed in Elephant v0.8.x
  DeprecationWarning)


In [2]:

df, df_monster, index_cols, firing_rates = generate_fake_spikes(n_trials=6,
                                                                n_neurons=8,
                                                                n_mice=4,
                                                                dur=7, )

# Exploratory plot without any fitting

In [3]:
from bayes_window.visualization import fake_spikes_explore
charts=fake_spikes_explore(df,df_monster,index_cols)
[chart.display() for chart in charts];

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.LayerChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.LayerChart(...)

# Estimate with mouse>neuron

In [4]:
from bayes_window import workflow
from importlib import reload
reload(workflow)

<module 'bayes_window.workflow' from '/home/m/mmy/bayes-window/bayes_window/workflow.py'>

In [5]:
workflow.estimate_slope(df,y='isi', levels=('stim', 'mouse', 'neuron'), hold_for_facet=False)

sample: 100%|██████████| 1100/1100 [00:07<00:00, 144.78it/s, 63 steps of size 9.88e-02. acc. prob=0.93]


n(Divergences) = 0


alt.LayerChart(...)

# Estimate with neuron>mouse

In [6]:

workflow.estimate_slope(df,y='isi', levels=('stim', 'neuron','mouse'), hold_for_facet=False)

sample: 100%|██████████| 1100/1100 [00:09<00:00, 118.10it/s, 383 steps of size 2.42e-02. acc. prob=0.91]


n(Divergences) = 0


alt.LayerChart(...)

TODO Plot per trial, too
Maybe not change but rather intercepts only

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y='isi'
levels=('stim', 'mouse', 'neuron')
df['combined_condition'] = df[levels[0]].astype('str')
for level in levels[1:]:
    df['combined_condition']+=df[level].astype('str')

# Transform conditions to integers as required by numpyro:
df['combined_condition'] = le.fit_transform(df['combined_condition'])
# Keep key for later use
# key = dict(zip(range(len(le.classes_)), le.classes_))

# Estimate model
trace = fit_numpyro(y=df[y].values,
                    treat=df['combined_condition'].values,
                    progress_bar=True,
                    model=models.model_single_lognormal,
                    n_draws=100, num_chains=1)

sample: 100%|██████████| 1100/1100 [00:05<00:00, 197.58it/s, 15 steps of size 1.73e-01. acc. prob=0.93]


n(Divergences) = 0


In [8]:
# Add data back
df_both = add_data_to_posterior(df,
                                trace=trace,
                                y=y,
                                index_cols=levels[:3],
                                condition_name=levels[0],
                                b_name='mu_per_condition',  # for posterior
                                group_name=levels[2]  # for posterior
                                )

In [11]:
import altair as alt
x=levels[2]
color=levels[1]
# Make bayes
points = alt.Chart().mark_point(filled=True, color='black').encode(
    y=alt.Y('Bayes condition mean:Q', scale=alt.Scale(zero=False)),
    x=x
)

line = alt.Chart().mark_line(color='black').encode(
    y=alt.Y('Bayes condition mean:Q', scale=alt.Scale(zero=False)),
    x=x
)

error_bars = alt.Chart().mark_rule().encode(
    x=x,
    y=alt.Y('Bayes condition CI0:Q',  scale=alt.Scale(zero=False)),
    y2='Bayes condition CI1:Q',
)

c2 = ( points + line + error_bars)
alt.layer(c2, data=df_both)

alt.LayerChart(...)